In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from typing import Union, List, Tuple, Dict

In [ ]:
def extraer_series_embalses(file: Path, ref_ceh: Union[str, List[str]] = None) -> pd.DataFrame: #, plot=False, xlim=None):
    """Extrae las series del Anuario de Aforos del CEDEX del embalse indicado
    
    Entrada:
    --------
    archivo:  str. Ruta y nombre del archivo con los datos de los embalses del ROEA
    estacion: int. Código de la estacion ROEA del embalse
    plot:     boolean. Si se quiere mostrar la gráfica de las series
    xlim:     list. Fechas de inicio y fin de la gráfica
    
    Salida:
    -------
    data:     DataFrame. Serie diaria de caudal de entrada, salida (m³/s) y reserva (hm³) en el embalse.
    """
    
    # SERIE
    # -----
    # cargar datos
    data = pd.read_csv(archivo, parse_dates=True)
    # seleccionar datos de la estación
    data = data.loc[data['ref_ceh'] == estacion, :]
    # crear índice de fecha
    data.index = pd.to_datetime(data.fecha, dayfirst=True)
    # calcular caudal de entrada
    data['entrada'] = np.nan
    for day1 in data.index[:-1]:
        day2 = day1 + datetime.timedelta(days=1)
        AV = data.loc[day2, 'reserva'] - data.loc[day1, 'reserva'] # hm³
        AV /= 0.0864 # m³/s
        if data.loc[day1, 'tipo'] == 1:
            data.loc[day2, 'entrada'] = AV + data.loc[day2, 'salida']
        elif data.loc[day1, 'tipo'] == 2:
            data.loc[day1, 'entrada'] = AV + data.loc[day1, 'salida']
    # redondear
    data.entrada = data.entrada.round(2)
    # eliminar columnas innecesarias
    data.drop(['fecha', 'ref_ceh', 'tipo'], axis=1, inplace=True)
    
    # GRÁFICA
    # -------
    if plot == True:
        fig, ax1 = plt.subplots(figsize=(15, 4))
        # caudal
        ax1.tick_params(labelsize=11)
        ax1.plot(data.entrada, 'steelblue', lw=.5, label='entrada')
        ax1.plot(data.salida, 'indianred', lw=.5, label='salida')
        qmax = np.ceil(data[['entrada', 'salida']].max().max() / 100) * 100
        ylim1 = (-.2 * qmax, qmax)
        if xlim is None:
            ax1.set(xlim=(xlim), ylim=ylim1)
        else:
            ax1.set(xlim(data.index[0], data.index[-1]), ylim=ylim1)
        yticks1 = np.arange(-.2 * qmax, qmax + 1, qmax / 5)
        ax1.set_yticks = yticks1
        ax1.set_yticklabels = yticks1
        ax1.set_ylabel('Q (m³/s)', fontsize=13)
        # volumen
        ax2 = ax1.twinx()
        ax2.tick_params(labelsize=11)
        ax2.fill_between(data.index, data.reserva, color='grey', alpha=.25, label='reserva')
        vmax = np.ceil(data.reserva.max() / 50) * 50
        ylim2 = (-.2 * vmax, vmax)
        if xlim is None:
            ax2.set(xlim=(xlim), ylim=ylim2)
        else:
            ax2.set(xlim(data.index[0], data.index[-1]), ylim=ylim2)
        yticks2 = np.arange(-.2 * vmax, vmax + 1, vmax / 5)
        ax2.set_yticks = yticks2
        ax2.set_yticklabels = yticks2
        ax2.set_ylabel('V (hm³)', fontsize=13)
        fig.legend(loc=8, ncol=3, fontsize=13);

    return data